In [5]:
import requests
import pandas as pd
import yaml


def run_query(query):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': 'BQYz1346Js0w2ccp4fytxBHdFmkTbhsm'}
    request = requests.post('https://graphql.bitquery.io/',
                            json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}.      {}'.format(request.status_code,
                        query))

#### This query returns the total trade volumes and trades on Pancakeswap

In [39]:
query = """
{
  ethereum(network: bsc) {
    dexTrades(
      exchangeName: {in: ["Pancake v2"]}
      time: {since: "2022-01-01"}
    ) {
      count
      tradeAmount(in: USD)
    }
  }
}
"""
result = run_query(query)  # Execute the query
print ('Result - {}'.format(result))

Result - {'data': {'ethereum': {'dexTrades': None}}, 'errors': [{'message': 'ActiveRecord::ActiveRecordError: Response code: 500:\nCode: 241, e.displayText() = DB::Exception: Memory limit (for query) exceeded: would use 30.93 GiB (attempt to allocate chunk of 2147484176 bytes), maximum: 29.80 GiB (version 20.8.11.17 (official build))\n', 'locations': [{'line': 4, 'column': 5}], 'path': ['ethereum', 'dexTrades'], 'error_type': 'server'}]}


In [36]:
# Convert the result to a dataframe
df = pd.json_normalize(result['data']['ethereum']['dexTrades'])
print(df)

     count   tradeAmount
0  1464450  6.852427e+08


#### This query returns the top most exchanged tokens on Pancakeswap this year

In [22]:
query = """
{

  ethereum(network: bsc) {
    dexTrades(
      smartContractAddress: {is: "0x05faf555522Fa3F93959F86B41A3808666093210"}
      time: {since: "2022-01-01"}
    ) {
      baseCurrency {
        address
        decimals
        name
        symbol
        tokenId
        tokenType
      }
      buyAmount(in: USD)
      tradeAmount(in: USD)
    }
  }

}
"""
result = run_query(query)  # Execute the query
print ('Result - {}'.format(result))

Result - {'data': {'ethereum': {'dexTrades': [{'baseCurrency': {'address': '0x23396cf899ca06c4472205fc903bdb4de249d6fc', 'decimals': 18, 'name': 'Wrapped UST Token', 'symbol': 'UST', 'tokenId': '', 'tokenType': 'ERC20'}, 'buyAmount': 233010514.8295463, 'tradeAmount': 467645748.5565017}, {'baseCurrency': {'address': '0xe9e7cea3dedca5984780bafc599bd69add087d56', 'decimals': 18, 'name': 'BUSD Token', 'symbol': 'BUSD', 'tokenId': '', 'tokenType': 'ERC20'}, 'buyAmount': 233010514.82954678, 'tradeAmount': 467645748.55649763}]}}}


In [23]:
# Convert the result to a dataframe
df = pd.json_normalize(result['data']['ethereum']['dexTrades'])
print(df)

      buyAmount   tradeAmount                        baseCurrency.address  \
0  2.330105e+08  4.676457e+08  0x23396cf899ca06c4472205fc903bdb4de249d6fc   
1  2.330105e+08  4.676457e+08  0xe9e7cea3dedca5984780bafc599bd69add087d56   

   baseCurrency.decimals  baseCurrency.name baseCurrency.symbol  \
0                     18  Wrapped UST Token                 UST   
1                     18         BUSD Token                BUSD   

  baseCurrency.tokenId baseCurrency.tokenType  
0                                       ERC20  
1                                       ERC20  


#### Total DCIP (Decentralized community investment protocol) trade volume

In [24]:
query = """
{
   ethereum (network: bsc){
    dexTrades( options: {desc: "count"},
      smartContractAddress: 
      {is:"0x4799b0d36b421df620dafebde3ba19c2c2c2fc5c"},
      date: {since:"2020-12-12", till: "2022-12-12"}
    ) {
      count
      tradeAmount(in:USD)
    }
  }
}
"""
result = run_query(query)  # Execute the query
print ('Result - {}'.format(result))

Result - {'data': {'ethereum': {'dexTrades': [{'count': 53472, 'tradeAmount': 16872882.11736385}]}}}


In [25]:
# Convert the result to a dataframe
df = pd.json_normalize(result['data']['ethereum']['dexTrades'])
print(df)

   count   tradeAmount
0  53472  1.687288e+07


#### Trade amount between BNB and USDT in the last 7 days

In [30]:
query = """
{
ethereum (network: bsc){
dexTrades(options: {limit :10,desc: "date.date"},
exchangeName: {is: "Pancake v2"} 
baseCurrency: {is: "0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c"}
quoteCurrency: {is: "0x55d398326f99059ff775485246999027b3197955"}
){
count
tradeAmount(in:USD)
date{
date(format: "%Y-%m-%d")
}}
}}






"""
result = run_query(query)  # Execute the query
print ('Result - {}'.format(result))


Result - {'data': {'ethereum': {'dexTrades': [{'count': 17749, 'tradeAmount': 12890932.539569177, 'date': {'date': '2022-05-17'}}, {'count': 84672, 'tradeAmount': 49344692.942646265, 'date': {'date': '2022-05-16'}}, {'count': 89041, 'tradeAmount': 47089693.832172155, 'date': {'date': '2022-05-15'}}, {'count': 110776, 'tradeAmount': 60058548.065879814, 'date': {'date': '2022-05-14'}}, {'count': 121675, 'tradeAmount': 81626206.475333, 'date': {'date': '2022-05-13'}}, {'count': 136554, 'tradeAmount': 183108344.44585732, 'date': {'date': '2022-05-12'}}, {'count': 117552, 'tradeAmount': 196939411.05205688, 'date': {'date': '2022-05-11'}}, {'count': 102100, 'tradeAmount': 123439638.34973899, 'date': {'date': '2022-05-10'}}, {'count': 111592, 'tradeAmount': 139763224.12998426, 'date': {'date': '2022-05-09'}}, {'count': 108292, 'tradeAmount': 73009793.21477877, 'date': {'date': '2022-05-08'}}]}}}


In [31]:
# Convert the result to a dataframe
df = pd.json_normalize(result['data']['ethereum']['dexTrades'])
print(df)

    count   tradeAmount   date.date
0   17749  1.289093e+07  2022-05-17
1   84672  4.934469e+07  2022-05-16
2   89041  4.708969e+07  2022-05-15
3  110776  6.005855e+07  2022-05-14
4  121675  8.162621e+07  2022-05-13
5  136554  1.831083e+08  2022-05-12
6  117552  1.969394e+08  2022-05-11
7  102100  1.234396e+08  2022-05-10
8  111592  1.397632e+08  2022-05-09
9  108292  7.300979e+07  2022-05-08
